In [1]:
import joblib 
import os
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from experiments_helpers import ExperimentResults

if 'src' in os.getcwd():
    os.chdir('..')
    print(os.getcwd())

folder = 'results2'
name = 'torch-fico-gs'

idx = 0
es_base: ExperimentResults = ExperimentResults.load_results_from_file(f'{folder}/{name}_{idx}.joblib')

es_base.pretty_print_robust_vs_base()

/home/ignacy/research/robust-cf/src


2024-02-29 19:51:09.479776: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-29 19:51:09.507576: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


/home/ignacy/research/robust-cf
ExperimentResults with 18 metrics and 3 artifacts.
############################## Metrics ##############################
------------------------- Base metrics -------------------------
validity: 1.00 (std: 0.00)
proximityL1: 0.60 (std: 0.44)
lof: -1.26 (std: 0.36)
cf_counterfactual_stability: 0.03 (std: 0.12)
generation_time: 0.01 (std: 0.01)
------------------------- Base metrics 2 -------------------------
validity_2: 0.19 (std: 0.39)
proximityL1_2: 0.60 (std: 0.44)
lof_2: -1.26 (std: 0.36)
cf_counterfactual_stability_2: 0.44 (std: 0.22)
------------------------- Robust metrics -------------------------
robust_validity: nan (std: nan)
robust_proximityL1: nan (std: nan)
robust_lof: nan (std: nan)
robust_cf_counterfactual_stability: nan (std: nan)
robust_generation_time: 93.63 (std: 48.67)
------------------------- Robust metrics 2 -------------------------
robust_validity_2: nan (std: nan)
robust_proximityL1_2: nan (std: nan)
robust_lof_2: nan (std: na